In [1]:
import os

import sqlite3 as sq
from sqlalchemy import create_engine, inspect, Table, MetaData
from sqlalchemy.orm import sessionmaker

import pandas as pd


In [2]:
mta_data_engine = create_engine("sqlite:///mta_data.db")


In [3]:
def insert_table_from_csv(csv_list, engine):
    for file in csv_list:
        with open(file, 'r') as f:
            data = pd.read_csv(f)
            data.columns = data.columns.str.strip()
        data.to_sql(os.path.splitext(file)[0], con=mta_data_engine, index=False, if_exists='replace')
    

In [4]:
insert_table_from_csv(['mta_stations_loc.csv', 'mta_complex_id.csv'], mta_data_engine)


In [5]:
insp = inspect(mta_data_engine)
print(insp.get_table_names())


['mta_complex_id', 'mta_data', 'mta_stations_loc']


In [9]:
pd.read_sql('''
            SELECT a.booth, a.unit, a.scp, a.station, a.linename, a.division, 
              MAX(a.entries)-MIN(a.entries) AS NET_ENTRIES,
              MAX(a.exits)-MIN(a.exits) AS NET_EXITS,
              CAST(b.complex_id AS int) AS COMPLEX_ID, 
              c."GTFS Latitude" AS LAT, c."GTFS Longitude" AS LON
            FROM mta_data a
            JOIN mta_complex_id b
            JOIN mta_stations_loc c
            ON a.booth = b.booth AND a.unit = b.remote AND b.complex_ID = c."Complex ID"
            WHERE a.date >= '06/01/2017' 
              AND a.date <= '08/31/2017' 
              AND a.desc = 'REGULAR'
            GROUP BY a.booth, a.unit, a.scp;
            ''', 
            mta_data_engine)

,BOOTH,UNIT,SCP,STATION,LINENAME,DIVISION,NET_ENTRIES,NET_EXITS,COMPLEX_ID,LAT,LON
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,109120,35500,613,40.762526,-73.967967
1,A002,R051,02-00-01,59 ST,NQR456W,BMT,84747,21705,613,40.762526,-73.967967
2,A002,R051,02-03-00,59 ST,NQR456W,BMT,44528,150602,613,40.762526,-73.967967
3,A002,R051,02-03-01,59 ST,NQR456W,BMT,106804,152432,613,40.762526,-73.967967
4,A002,R051,02-03-02,59 ST,NQR456W,BMT,107756,123008,613,40.762526,-73.967967
...,...,...,...,...,...,...,...,...,...,...,...
4233,S102,R165,00-03-00,TOMPKINSVILLE,1,SRT,15801,0,502,40.636949,-74.074835
4234,S102,R165,00-03-01,TOMPKINSVILLE,1,SRT,10504,0,502,40.636949,-74.074835
4235,S102,R165,00-03-02,TOMPKINSVILLE,1,SRT,6879,0,502,40.636949,-74.074835
4236,S102,R165,00-05-00,TOMPKINSVILLE,1,SRT,28,0,502,40.636949,-74.074835
